In [48]:
# Instalamos torchtext que nos facilitará la vida en el pre-procesamiento del formato ConLL.
# !pip install -U torchtext==0.10.0

# Librerias

In [49]:
import random
import os, shutil
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gzip
import os
import shutil
import requests

from operator import attrgetter
from torchtext import vocab, datasets ,data
#from torchtext.legacy import data #, datasets
from seqeval.metrics import f1_score, precision_score, recall_score

In [50]:
# Garantizar reproducibilidad de los experimentos
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#### **Obtener datos**

Descargamos los datos de entrenamiento, validación y prueba en nuestro directorio de trabajo

In [51]:
#%%capture

# !wget https://github.com/dccuchile/CC6205/releases/download/v1.0/train.txt -nc # Dataset de Entrenamiento
# !wget https://github.com/dccuchile/CC6205/releases/download/v1.0/dev.txt -nc    # Dataset de Validación (Para probar y ajustar el modelo)
# !wget https://github.com/dccuchile/CC6205/releases/download/v1.0/test.txt -nc  # Dataset de la Competencia. Estos datos solo contienen los tokens. ¡¡SON LOS QUE DEBEN SER PREDICHOS!!

In [52]:
# Primer Field: TEXT. Representan los tokens de la secuencia
TEXT = data.Field(lower=False) 

# Segundo Field: NER_TAGS. Representan los Tags asociados a cada palabra.
NER_TAGS = data.Field(unk_token=None)
fields = (("text", TEXT), ("nertags", NER_TAGS))

print(fields)

(('text', <torchtext.data.field.Field object at 0x7fcab250d150>), ('nertags', <torchtext.data.field.Field object at 0x7fcab250ee90>))


####  **SequenceTaggingDataset**

`SequenceTaggingDataset` es una clase de torchtext diseñada para contener datasets de sequence labeling. Los ejemplos que se guarden en una instancia de estos serán arreglos de palabras asociados con sus respectivos tags.

Por ejemplo, para Part-of-speech tagging:

[I, love, PyTorch, .] estará asociado con [PRON, VERB, PROPN, PUNCT]


La idea es que usando los fields que definimos antes, le indiquemos a la clase cómo cargar los datasets de prueba, validación y test.

In [53]:
train_data, valid_data, test_data = datasets.SequenceTaggingDataset.splits(
    path="./",
    train="corpus_recetas_train.txt",
    validation="corpus_recetas_val.txt",
    test="corpus_recetas_test.txt",
    fields=fields,
    encoding="utf-8",
    separator=" "
)

In [54]:
# valid_data = train_data[int(len(train_data)*0.7):int(len(train_data)*0.8)]
# test_data = train_data[int(len(train_data)*0.8):len(train_data)]
# train_data = train_data[0:int(len(train_data)*0.7)]

In [55]:
print(f"Numero de ejemplos de entrenamiento: {len(train_data)}")
print(f"Número de ejemplos de validación: {len(valid_data)}")
print(f"Número de ejemplos de test (competencia): {len(test_data)}")

Numero de ejemplos de entrenamiento: 500
Número de ejemplos de validación: 250
Número de ejemplos de test (competencia): 250


Visualizemos un ejemplo

In [56]:
random_item_idx = random.randint(0, len(train_data))
random_example = train_data.examples[random_item_idx]
list(zip(random_example.text, random_example.nertags))

[('ZOPICLONA', 'B-ACTVPRNCP'),
 ('7,5', 'O'),
 ('MG', 'O'),
 ('COMPRIMIDO', 'B-ADMIN'),
 ('7,5', 'O'),
 ('MG', 'O'),
 ('ORAL', 'B-ADMIN'),
 ('diaria', 'O'),
 ('0-0-7.5', 'O'),
 ('mg', 'O')]

#### **Construir los vocabularios para el texto y las etiquetas**

Los vocabularios son los objetos que contienen todos los tokens (de entrenamiento) posibles para ambos fields. El siguiente paso consiste en construirlos. Para esto, hacemos uso del método `Field.build_vocab` sobre cada uno de nuestros `fields`. 

In [57]:
TEXT.build_vocab(train_data)
NER_TAGS.build_vocab(train_data)

In [58]:
print(f"Tokens únicos en TEXT: {len(TEXT.vocab)}")
print(f"Tokens únicos en NER_TAGS: {len(NER_TAGS.vocab)}")

Tokens únicos en TEXT: 683
Tokens únicos en NER_TAGS: 10


In [59]:
#Veamos las posibles etiquetas que hemos cargado:
NER_TAGS.vocab.itos

['<pad>',
 'O',
 'B-ADMIN',
 'B-ACTVPRNCP',
 'I-ADMIN',
 'I-PERIODICITY',
 'I-DURATION',
 'B-PERIODICITY',
 'I-ACTVPRNCP',
 'B-DURATION']

Observen que ademas de los tags NER, tenemos \<pad\>, el cual es generado por el dataloader para cumplir con el padding de cada oración.

Veamos ahora los tokens mas frecuentes y especiales:

In [60]:
# Tokens mas frecuentes (Será necesario usar stopwords, eliminar símbolos o nos entregan información (?) )
TEXT.vocab.freqs.most_common(10)

[('ML', 352),
 ('MG', 332),
 ('ORAL', 284),
 ('cada', 279),
 ('COMPRIMIDO', 273),
 ('horas', 267),
 ('1', 235),
 ('SOLUCIÓN', 213),
 ('INYECTABLE', 210),
 ('AMPOLLA', 197)]

In [61]:
# Seteamos algunas variables que nos serán de utilidad mas adelante...
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

PAD_TAG_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
O_TAG_IDX = NER_TAGS.vocab.stoi['O']

#### **Frecuencia de los Tags**

Visualizemos rápidamente las cantidades y frecuencias de cada tag:

In [62]:
def tag_percentage(tag_counts):
    
    total_count = sum([count for tag, count in tag_counts])
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]
  
    return tag_counts_percentages

print("Tag Ocurrencia Porcentaje\n")

for tag, count, percent in tag_percentage(NER_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t{count}\t{percent*100:4.1f}%")

Tag Ocurrencia Porcentaje

O	4122	54.4%
B-ADMIN	789	10.4%
B-ACTVPRNCP	622	 8.2%
I-ADMIN	602	 7.9%
I-PERIODICITY	554	 7.3%
I-DURATION	292	 3.9%
B-PERIODICITY	277	 3.7%
I-ACTVPRNCP	177	 2.3%
B-DURATION	146	 1.9%


#### **Configuramos pytorch y dividimos los datos.**

Importante: si tienes problemas con la ram de la gpu, disminuye el tamaño de los batches

In [63]:
BATCH_SIZE = 32 #16  # disminuir si hay problemas de ram.

# Usar cuda si es que está disponible.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

# Dividir datos entre entrenamiento y test. Si van a hacer algún sort no puede ser sobre
# el conjunto de testing ya que al hacer sus predicciones sobre el conjunto de test sin etiquetas
# debe conservar el orden original para ser comparado con los golden_labels. 

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,
)

Using cuda


#### **Métricas de evaluación**

Además, definiremos las métricas que serán usadas tanto para la competencia como para evaluar el modelo: `precision`, `recall` y `micro f1-score`.
**Importante**: Noten que la evaluación solo se hace para las Named Entities (sin contar 'O'), toda esta funcionalidad nos la entrega la librería seqeval, pueden revisar más documentación aquí: https://github.com/chakki-works/seqeval. No utilicen el código entregado por sklearn para calcular las métricas ya que esta lo hace a nivel de token y no a nivel de entidad.

In [64]:
# Definimos las métricas
def calculate_metrics(preds, y_true, pad_idx=PAD_TAG_IDX, o_idx=O_TAG_IDX):
    """
    Calcula precision, recall y f1 de cada batch.
    """

    # Obtener el indice de la clase con probabilidad mayor. (clases)
    y_pred = preds.argmax(dim=1, keepdim=True)

    # filtramos <pad> para calcular los scores.
    mask = [(y_true != pad_idx)]
    y_pred = y_pred[mask]
    y_true = y_true[mask]

    # traemos a la cpu
    y_pred = y_pred.view(-1).to('cpu').numpy()
    y_true = y_true.to('cpu').numpy()
    y_pred = [[NER_TAGS.vocab.itos[v] for v in y_pred]]
    y_true = [[NER_TAGS.vocab.itos[v] for v in y_true]]
    
    # calcular scores
    f1 = f1_score(y_true, y_pred, mode='strict')
    precision = precision_score(y_true, y_pred, mode='strict')
    recall = recall_score(y_true, y_pred, mode='strict')

    return precision, recall, f1

-------------------

### **Modelo Baseline**

Teniendo ya cargado los datos, toca definir nuestro modelo. Este baseline tendrá una capa de embedding, unas cuantas LSTM y una capa de salida y usará dropout en el entrenamiento.

Este constará de los siguientes pasos: 

1. Definir la clase que contendrá la red.
2. Definir los hiperparámetros e inicializar la red. 
3. Definir el número de épocas de entrenamiento
4. Definir la función de loss.



Recomendamos que para experimentar, encapsules los modelos en una sola variable y luego la fijes en model para entrenarla

In [65]:
# Definir la red
class NER_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding(input_dim,
                                      embedding_dim,
                                      padding_idx=pad_idx,
                                      )

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        
        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

#### **Hiperparámetros de la red**



In [66]:
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.5
BIDIRECTIONAL = False

# Creamos nuestro modelo.
baseline_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

baseline_model_name = 'baseline'  # nombre que tendrá el modelo guardado...

In [67]:
baseline_n_epochs = 10

#### Definimos la función de loss

In [68]:
# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
baseline_criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

--------------------
### Modelo 1

En estas secciones pueden implementar nuevas redes al modificar los hiperparámetros, la cantidad de épocas de entrenamiento, el tamaño de los batches, loss, optimizador, etc... como también definir nuevas arquitecturas de red (mediante la creación de clases nuevas)


Al final de estas, hay 4 variables, las cuales deben setear con los modelos, épocas de entrenamiento, loss y optimizador que deseen probar.


In [69]:
# Caergamos Glove o fast text
FASTTEXT_FILE = "glove300d.vec"
# Se descargan vectores glove o fasttext del github del dcc
# https://github.com/dccuchile/spanish-word-embeddings

if not os.path.exists(FASTTEXT_FILE):
    print(f"Descargando {FASTTEXT_FILE}")
    url = "http://dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.vec.gz"
    #url = "https://s06.imfd.cl/04/fasttext-sbwc.vec.gz"
    response = requests.get(url, stream=True)
    try:
        with gzip.open(response.raw, "rb") as f_in:
            with open(FASTTEXT_FILE, "wb") as f_out:
                # Funcion para copiar de un file-like object a otro
                shutil.copyfileobj(f_in, f_out)
    except Exception as e:
        os.remove(FASTTEXT_FILE)
        raise e

 #dimensión es de 300 y tiene 855,380 vectores pre-entrenados. 

In [70]:
embeddings = vocab.Vectors(FASTTEXT_FILE)


In [71]:
TEXT.vocab.set_vectors(*attrgetter("stoi", "vectors", "dim")(embeddings))
#se asocian los tokens de text a los vectores del embedding


In [72]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,
)

Using cuda


In [73]:
class Modelo1_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding.from_pretrained(
            embedding_weights.clone(), freeze = False)
        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)
        

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        
        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

In [74]:
#Parametros para el modelo 1
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 196
OUTPUT_DIM = len(NER_TAGS.vocab)
embedding_weights = TEXT.vocab.vectors

N_LAYERS = 3
DROPOUT = 0.15
BIDIRECTIONAL = True

TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]


In [75]:
modelo_1 = Modelo1_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, 
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

model_name_1 = 'Modelo_1'
n_epochs_1 = 100 #no importa porq hay early stop
criterion_1 = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

---------------

### Modelo 2

In [76]:
BATCH_SIZE = 62 # disminuir si hay problemas de ram.

# Usar cuda si es que está disponible.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

# Dividir datos entre entrenamiento y test. Si van a hacer algún sort no puede ser sobre
# el conjunto de testing ya que al hacer sus predicciones sobre el conjunto de test sin etiquetas
# debe conservar el orden original para ser comparado con los golden_labels. 

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,
)

Using cuda


In [77]:
class Modelo2_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding.from_pretrained(embedding_weights.clone(), freeze = False)

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)
        #self.relu = nn.ReLU()

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        
        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

In [78]:
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 196  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 2  # número de capas.
DROPOUT = 0.4
BIDIRECTIONAL = True
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]

In [79]:
modelo_2 = Modelo2_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, 
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

model_name_2 = 'Modelo_2'
n_epochs_2 = 30 #no importa porq hay early stop
criterion_2 = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

---------------


### Modelo 3

In [80]:
# Para crear la red debemos heredar desde nn.Module
class GruNet(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

      super().__init__()

      # Capa de embedding
      self.embedding = nn.Embedding(input_dim,
                                    embedding_dim,
                                    padding_idx=pad_idx,
                                    )
      
      
      # Capa GRU
      self.gru = nn.GRU(embedding_dim, hidden_dim, n_layers, batch_first=True, dropout = dropout if n_layers > 1 else 0, bidirectional=bidirectional)

      # Capa de salida
      self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)


      #self.relu = nn.ReLU()

      # Dropout
      self.dropout = nn.Dropout(dropout)




    # Definimos las operaciones de las capas sobre el input en el forward.
    def forward(self, text): 

      embedded = self.embedding(text)

      outputs, hidden = self.gru(embedded)

      predictions = self.fc(self.dropout(outputs))


      return predictions


In [81]:
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 2  # número de capas.
DROPOUT = 0.3
BIDIRECTIONAL = False

# Creamos nuestro modelo.
modelo_3 = GruNet(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

model_name_3 = 'Gru_Model'  # nombre que tendrá el modelo guardado...

n_epochs_3 = 10


# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
criterion_3 = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)



In [82]:
a ='123'
a[:2]

'12'

In [83]:
def function(fecha1,fecha2):
    Año_1=int(str(fecha1)[:2])
    Mes_1=int(str(fecha1)[-2:])
    Año_2=int(str(fecha2)[:2])
    Mes_2=int(str(fecha2)[-2:])

    dif=12*(Año_2-Año_1)+(Mes_2-Mes_1)
    return dif 



#### **Inicializamos la red**

Iniciamos los pesos de la red de forma aleatoria (Usando una distribución normal).


In [84]:
model = baseline_model
model_name = baseline_model_name
criterion = baseline_criterion
n_epochs = baseline_n_epochs

# Optimizador
optimizer = optim.Adam(model.parameters())

# # Enviamos el modelo y la loss a cuda (en el caso en que esté disponible)
# model = model.to(device)
# criterion = criterion.to(device)

In [85]:
def init_weights(m):
    # Inicializamos los pesos como aleatorios
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1) 
        
    # Seteamos como 0 los embeddings de UNK y PAD.
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
        
model.apply(init_weights)

NER_RNN(
  (embedding): Embedding(683, 300, padding_idx=1)
  (lstm): LSTM(300, 256, num_layers=3, dropout=0.5)
  (fc): Linear(in_features=256, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [86]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

El modelo actual tiene 1,831,534 parámetros entrenables.


Notar que definimos los embeddings que representan a \<unk\> y \<pad\>  como [0, 0, ..., 0]

#### **Definimos el entrenamiento de la red**


In [87]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.train()
    # Por cada batch del iterador de la época:
    for batch in iterator:

        # Extraemos el texto y los tags del batch que estamos procesado
        text = batch.text
        tags = batch.nertags

        # Reiniciamos los gradientes calculados en la iteración anterior
        optimizer.zero_grad()

        #text = [sent len, batch size]

        # Predecimos los tags del texto del batch.
        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]

        # Reordenamos los datos para calcular la loss
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        #predictions = [sent len * batch size, output dim]
        # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
        loss = criterion(predictions, tags)
        
        # Calculamos el accuracy
        precision, recall, f1 = calculate_metrics(predictions, tags)

        # Calculamos los gradientes
        loss.backward()

        # Actualizamos los parámetros de la red
        optimizer.step()

        # Actualizamos el loss y las métricas
        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

#### **Definimos la función de evaluación**

In [88]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()

    # Indicamos que ahora no guardaremos los gradientes
    with torch.no_grad():
        # Por cada batch
        for batch in iterator:

            text = batch.text
            tags = batch.nertags

            # Predecimos
            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
            loss = criterion(predictions, tags)

            # Calculamos las métricas
            precision, recall, f1 = calculate_metrics(predictions, tags)

            # Actualizamos el loss y las métricas
            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

In [89]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


#### **Entrenamiento de la red**

En este cuadro de código ejecutaremos el entrenamiento de la red.
Para esto, primero definiremos el número de épocas y luego por cada época, ejecutaremos `train` y `evaluate`.

**Importante: Reiniciar los pesos del modelo**

Si ejecutas nuevamente esta celda, se seguira entrenando el mismo modelo una y otra vez. 
Para reiniciar el modelo se debe ejecutar nuevamente la celda que contiene la función `init_weights`



In [92]:
def train_model(model, model_name, criterion, optimizer, n_epochs):
  global device

  model = model.to(device)
  criterion = criterion.to(device)
  
  #Agregar early stop
  best_valid_loss = float('inf')
  best_train_loss = float('inf')

  arrayTrainLoss = []
  arrayValidLoss = []

  for epoch in range(n_epochs):

    start_time = time.time()

    # Recuerdo: train_iterator y valid_iterator contienen el dataset dividido en batches.

    # Entrenar
    train_loss, train_precision, train_recall, train_f1 = train(
        model, train_iterator, optimizer, criterion)

    # Evaluar (valid = validación)
    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
        model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    #Nuevo **
    arrayTrainLoss.append(train_loss)
    arrayValidLoss.append(valid_loss)

  #Acá nos aseguramos de que entrene al menos 4 épocas antes de detener el entrenamiento.
    if epoch < 5:
      if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        #nuevo
        best_train_loss = train_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )
      else :
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )

  #Early stop, ve que no vaya en aumento, según el avg de las últimas 4 épocas
    else:
      if train_loss < np.mean(arrayTrainLoss[-4:]) and valid_loss > np.mean(arrayValidLoss[-4:]):
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )
        print('Early Stop')
        break

      else:
        best_valid_loss = valid_loss
        #nuevo
        best_train_loss = train_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )
  return model, arrayTrainLoss, arrayValidLoss

## Entrenamiento Baseline

In [93]:
model = baseline_model
model_name = baseline_model_name
criterion = baseline_criterion
n_epochs = baseline_n_epochs

model, arrayTrainLoss, arrayValidLoss = train_model(model, model_name, criterion, optimizer, n_epochs)

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 1.745 | Train f1: 0.04 | Train precision: 0.12 | Train recall: 0.03
	 Val. Loss: 1.445 |  Val. f1: 0.00 |  Val. precision: 0.00 | Val. recall: 0.00
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 1.429 | Train f1: 0.03 | Train precision: 0.14 | Train recall: 0.02
	 Val. Loss: 1.301 |  Val. f1: 0.09 |  Val. precision: 0.24 | Val. recall: 0.06
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 1.260 | Train f1: 0.19 | Train precision: 0.37 | Train recall: 0.12
	 Val. Loss: 1.128 |  Val. f1: 0.24 |  Val. precision: 0.61 | Val. recall: 0.15
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 1.133 | Train f1: 0.30 | Train precision: 0.50 | Train recall: 0.21
	 Val. Loss: 1.000 |  Val. f1: 0.31 |  Val. precision: 0.56 | Val. recall: 0.21
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.969 | Train f1: 0.40 | Train precision: 0.60 | Train recall: 0.31
	 Val. Loss: 0.859 |  Val. f1: 0.43 |  Val. precision: 0.69 | Val. recall: 0.31
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.

In [94]:
# cargar el mejor modelo entrenado.
model.load_state_dict(torch.load('{}.pt'.format(model_name)))

# Limpiar ram de cuda
torch.cuda.empty_cache()

In [95]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model, valid_iterator, criterion)

print(f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}')

Val. Loss: 0.284 |  Val. f1: 0.89 | Val. precision: 0.96 | Val. recall: 0.83


## Entrenamiento Modelo 1

In [96]:
model1 = modelo_1
model_name1 = model_name_1
criterion1 = criterion_1
n_epochs1 = n_epochs_1

# Optimizador
optimizer1 = optim.Adam(model.parameters())
model1, arrayTrainLoss1, arrayValidLoss1 = train_model(model1, model_name1, criterion1, optimizer1, n_epochs1)

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 2.297 | Train f1: 0.12 | Train precision: 0.09 | Train recall: 0.21
	 Val. Loss: 2.296 |  Val. f1: 0.11 |  Val. precision: 0.08 | Val. recall: 0.19
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 2.297 | Train f1: 0.12 | Train precision: 0.08 | Train recall: 0.21
	 Val. Loss: 2.296 |  Val. f1: 0.11 |  Val. precision: 0.08 | Val. recall: 0.19
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 2.296 | Train f1: 0.12 | Train precision: 0.09 | Train recall: 0.21
	 Val. Loss: 2.296 |  Val. f1: 0.11 |  Val. precision: 0.08 | Val. recall: 0.19
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 2.297 | Train f1: 0.12 | Train precision: 0.09 | Train recall: 0.23
	 Val. Loss: 2.296 |  Val. f1: 0.11 |  Val. precision: 0.08 | Val. recall: 0.19
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 2.297 | Train f1: 0.14 | Train precision: 0.10 | Train recall: 0.24
	 Val. Loss: 2.296 |  Val. f1: 0.11 |  Val. precision: 0.08 | Val. recall: 0.19
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 2.

In [97]:
# cargar el mejor modelo entrenado.
model1.load_state_dict(torch.load('{}.pt'.format(model_name1)))

# Limpiar ram de cuda
torch.cuda.empty_cache()

In [98]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model1, valid_iterator, criterion)

print(f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}')

Val. Loss: 2.296 |  Val. f1: 0.11 | Val. precision: 0.08 | Val. recall: 0.19


## Entrenamiento Modelo 2

In [99]:
model2 = modelo_2
model_name2 = model_name_2
criterion2 = criterion_2
n_epochs2 = n_epochs_2

# Optimizador
#optimizer = optim.Adam(model.parameters(),lr=0.3)
optimizer2 = optim.SGD(model.parameters(), lr=0.3, momentum=0.9)

model2, arrayTrainLoss2, arrayValidLoss2 = train_model(model2, model_name2, criterion2, optimizer2, n_epochs2)

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 2.299 | Train f1: 0.08 | Train precision: 0.06 | Train recall: 0.13
	 Val. Loss: 2.301 |  Val. f1: 0.05 |  Val. precision: 0.04 | Val. recall: 0.09
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 2.299 | Train f1: 0.08 | Train precision: 0.06 | Train recall: 0.13
	 Val. Loss: 2.301 |  Val. f1: 0.05 |  Val. precision: 0.04 | Val. recall: 0.09
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 2.301 | Train f1: 0.08 | Train precision: 0.06 | Train recall: 0.13
	 Val. Loss: 2.301 |  Val. f1: 0.05 |  Val. precision: 0.04 | Val. recall: 0.09
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 2.300 | Train f1: 0.08 | Train precision: 0.06 | Train recall: 0.14
	 Val. Loss: 2.301 |  Val. f1: 0.05 |  Val. precision: 0.04 | Val. recall: 0.09
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 2.299 | Train f1: 0.07 | Train precision: 0.05 | Train recall: 0.12
	 Val. Loss: 2.301 |  Val. f1: 0.05 |  Val. precision: 0.04 | Val. recall: 0.09
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 2.

In [100]:
model2.load_state_dict(torch.load('{}.pt'.format(model_name2)))

# Limpiar ram de cuda
torch.cuda.empty_cache()

In [101]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model2, valid_iterator, criterion)

print(f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}')

Val. Loss: 2.301 |  Val. f1: 0.05 | Val. precision: 0.04 | Val. recall: 0.09


## Entrenamiento Modelo 3

In [102]:
model3 = modelo_3
model_name3 = model_name_3
criterion3 = criterion_3
n_epochs3 = n_epochs_3

# Optimizador
optimizer3 = optim.Adam(model3.parameters())

model3, arrayTrainLoss3, arrayValidLoss3 = train_model(model3, model_name3, criterion3, optimizer3, n_epochs3)

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 1.765 | Train f1: 0.13 | Train precision: 0.39 | Train recall: 0.10
	 Val. Loss: 1.412 |  Val. f1: 0.36 |  Val. precision: 0.57 | Val. recall: 0.28
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 1.121 | Train f1: 0.48 | Train precision: 0.66 | Train recall: 0.39
	 Val. Loss: 1.009 |  Val. f1: 0.54 |  Val. precision: 0.68 | Val. recall: 0.46
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.682 | Train f1: 0.67 | Train precision: 0.78 | Train recall: 0.60
	 Val. Loss: 0.710 |  Val. f1: 0.65 |  Val. precision: 0.67 | Val. recall: 0.64
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.468 | Train f1: 0.78 | Train precision: 0.82 | Train recall: 0.74
	 Val. Loss: 0.583 |  Val. f1: 0.73 |  Val. precision: 0.73 | Val. recall: 0.73
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.374 | Train f1: 0.83 | Train precision: 0.87 | Train recall: 0.80
	 Val. Loss: 0.510 |  Val. f1: 0.81 |  Val. precision: 0.80 | Val. recall: 0.83
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.

In [103]:
model3.load_state_dict(torch.load('{}.pt'.format(model_name3)))

# Limpiar ram de cuda
torch.cuda.empty_cache()

In [104]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model3, valid_iterator, criterion)

print(f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}')

Val. Loss: 0.382 |  Val. f1: 0.85 | Val. precision: 0.82 | Val. recall: 0.88


### **Predecir datos para la competencia**

Ahora, a partir de los datos de **test** y nuestro modelo entrenado, vamos a predecir las etiquetas que serán evaluadas en la competencia.

In [106]:
def predict_labels(model, iterator, criterion, fields=fields):

    # Extraemos los vocabularios.
    text_field = fields[0][1]
    nertags_field = fields[1][1]
    tags_vocab = nertags_field.vocab.itos
    words_vocab = text_field.vocab.itos

    model.eval()

    predictions = []

    with torch.no_grad():

        for batch in iterator:

            text_batch = batch.text
            text_batch = torch.transpose(text_batch, 0, 1).tolist()

            # Predecir los tags de las sentences del batch
            predictions_batch = model(batch.text)
            predictions_batch = torch.transpose(predictions_batch, 0, 1)

            # por cada oración predicha:
            for sentence, sentence_prediction in zip(text_batch,
                                                     predictions_batch):
                for word_idx, word_predictions in zip(sentence,
                                                      sentence_prediction):
                    # Obtener el indice del tag con la probabilidad mas alta.
                    argmax_index = word_predictions.topk(1)[1]

                    current_tag = tags_vocab[argmax_index]
                    # Obtenemos la palabra
                    current_word = words_vocab[word_idx]

                    if current_word != '<pad>':
                        predictions.append([current_word, current_tag])
                predictions.append(['EOS', 'EOS'])


    return predictions




In [107]:
predictions = predict_labels(model, test_iterator, criterion)
predictions_m3 = predict_labels(model3, test_iterator, criterion3)

predictions

[['FENITOÍNA', 'B-ACTVPRNCP'],
 ['250', 'O'],
 ['MG/5', 'O'],
 ['ML', 'O'],
 ['SOLUCIÓN', 'B-ADMIN'],
 ['INYECTABLE', 'I-ADMIN'],
 ['FRASCO', 'O'],
 ['AMPOLLA', 'O'],
 ['5', 'O'],
 ['ML', 'O'],
 ['2', 'O'],
 ['UNIDAD', 'O'],
 ['INTRAMUSCULAR', 'O'],
 ['cada', 'B-PERIODICITY'],
 ['8', 'I-PERIODICITY'],
 ['horas', 'I-PERIODICITY'],
 ['EOS', 'EOS'],
 ['METRONIDAZOL', 'B-ACTVPRNCP'],
 ['500', 'O'],
 ['MG/100', 'O'],
 ['ML', 'O'],
 ['SOLUCIÓN', 'B-ADMIN'],
 ['INYECTABLE', 'I-ADMIN'],
 ['UNIDAD', 'O'],
 ['100', 'O'],
 ['ML', 'O'],
 ['1', 'O'],
 ['FRASCO', 'O'],
 ['PARENTERAL', 'O'],
 ['cada', 'B-PERIODICITY'],
 ['8', 'I-PERIODICITY'],
 ['horas', 'I-PERIODICITY'],
 ['durante', 'B-DURATION'],
 ['3', 'I-DURATION'],
 ['dias', 'I-DURATION'],
 ['EOS', 'EOS'],
 ['DEXAMETASONA', 'B-ACTVPRNCP'],
 ['4', 'O'],
 ['MG/ML', 'O'],
 ['SOLUCIÓN', 'B-ADMIN'],
 ['INYECTABLE', 'I-ADMIN'],
 ['AMPOLLA', 'O'],
 ['1', 'O'],
 ['ML', 'O'],
 ['2', 'O'],
 ['UNIDAD', 'O'],
 ['<unk>', 'O'],
 ['cada', 'B-PERIODICITY'],
 [

### **Generar el archivo para la submission**

No hay problema si aparecen unk en la salida. Estos no son relevantes para evaluarlos, usamos solo los tags.

In [108]:
if (os.path.isfile('./predictions.zip')):
    os.remove('./predictions.zip')

if (not os.path.isdir('./predictions')):
    os.mkdir('./predictions')

else:
    # Eliminar predicciones anteriores:
    shutil.rmtree('./predictions')
    os.mkdir('./predictions')

f = open('predictions/predictions.txt', 'w')
for i, (word, tag) in enumerate(predictions[:-1]):
    if word=='EOS' and tag=='EOS': f.write('\n')
    else: 
      if i == len(predictions[:-1])-1:
        f.write(word + ' ' + tag)
      else: f.write(word + ' ' + tag + '\n')

f.close()

a = shutil.make_archive('predictions', 'zip', './predictions')